## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import uuid
import re 

In [2]:
csvs = []

for n in range(1,13):
    filename = f'./bike-rental-starter-kit/data/JC-2016{str(n).zfill(2)}-citibike-tripdata.csv'
    df = pd.read_csv(filename)
    csvs.append(df)

weather = pd.read_csv('./bike-rental-starter-kit/data/newark_airport_2016.csv')

tripdata = pd.concat(csvs,ignore_index=True)

In [3]:

# Define a function to convert a string to snake case
def snake_case(s):
    return '_'.join(
        re.sub('([A-Z][a-z]+)', r' \1',
        re.sub('([A-Z]+)', r' \1',
        s.replace('-', ' '))).split()).lower()


In [4]:

## TRIPDATA ------------

# Parsing int64 in Int64 to normalize .info()
for col in tripdata.columns:
    if tripdata[col].dtypes == 'int64':
        tripdata[col] = tripdata[col].astype('Int64')

# Normalize str to title format 
for col in tripdata.columns:
    if tripdata[col].dtypes == 'object':
        tripdata[col] = tripdata[col].str.title()

# Parsing Float64 in Int64 to normalize .info()
tripdata['Birth Year'] = tripdata['Birth Year'].astype('Int64')

# Erasing birth date for people birth before 1916
tripdata[tripdata['Birth Year']<=1916] = None

# Mapping Int64 to str values
tripdata['Gender'] = tripdata['Gender'].map({2:'female', 1:'male'})

# Keeping only trip duration under 7days long
tripdata = tripdata[tripdata['Trip Duration'] <= 60*60*24*7]


# Creating an UUID for tripdata
tripdata['Ride ID'] = [str(uuid.uuid4()) for _ in range(len(tripdata))]

# Mapping User Type to Casual/Member

tripdata['User Type'] = tripdata['User Type'].map({'Subscriber':'Casual', 'Customer':'Member'})


tripdata.rename(columns={
    'Trip Duration':'Trip Duration', #KEEP
    'Start Time':'Started at',
    'Stop Time':'Ended at',
    'Start Station ID':'Start station ID',
    'Start Station Name':'Start station name',
    'Start Station Latitude':'Start latitude',
    'Start Station Longitude':'Start longitude',
    'End Station ID':'End station ID',
    'End Station Name':'End station name',
    'End Station Latitude':'End latitude',
    'End Station Longitude':'End Longitude',
    'Bike ID':'Bike ID', #KEEP 
    'User Type':'User Type', #KEEP Name -- Member or casual ride
    'Birth Year':'Birth Year', #KEEP
    'Gender':'Gender', #KEEP
    'Ride ID':'Ride ID' #Created
    # Missing -- Rideable type
})

weather.rename(columns={
    'station':'Station ID'
    ###
})

# Snake Casing the columns name
tripdata.columns = [snake_case(column) for column in tripdata.columns]

tripdata.Name = 'tripdata'

## WEATHER -------------

# Splitting Name/Region from intial Name column
weather['SPLIT'] = weather['NAME'].str.split(',')

if 'REGION' in weather.columns:
    pass
else:
    weather['NAME'] = weather['SPLIT'].str[0]
    weather['REGION'] = weather['SPLIT'].str[1]

# Delete SPLIT if exists
if 'SPLIT' in weather.columns:
    weather.drop('SPLIT', axis=1, inplace=True)
else:
    pass

# Normalize str to title format 
weather['NAME'] = weather['NAME'].str.title()

# Drop columns where all values are NaN
for col in weather.columns:
    if weather[col].isnull().sum() == len(weather[col]):
        weather.drop(col,axis=1, inplace=True)

# Adding Lat/Long for weather df
weather['Latitude'] = 40.689531
weather['Longitude'] =-74.174462

# Snake Casing the columns name
weather.columns = [snake_case(column) for column in weather.columns]

weather.Name = 'weather'



In [5]:
weather.head(10)

,station,name,date,awnd,prcp,snow,snwd,tavg,tmax,tmin,wdf2,wdf5,wsf2,wsf5,region,latitude,longitude
0,USW00014734,Newark Liberty International Airport,2016-01-01,12.75,0.00,0.0,0.0,41,43,34,270,280.0,25.9,35.1,NJ US,40.689531,-74.174462
1,USW00014734,Newark Liberty International Airport,2016-01-02,9.40,0.00,0.0,0.0,36,42,30,260,260.0,21.0,25.1,NJ US,40.689531,-74.174462
2,USW00014734,Newark Liberty International Airport,2016-01-03,10.29,0.00,0.0,0.0,37,47,28,270,250.0,23.9,30.0,NJ US,40.689531,-74.174462
3,USW00014734,Newark Liberty International Airport,2016-01-04,17.22,0.00,0.0,0.0,32,35,14,330,330.0,25.9,33.1,NJ US,40.689531,-74.174462
4,USW00014734,Newark Liberty International Airport,2016-01-05,9.84,0.00,0.0,0.0,19,31,10,360,350.0,25.1,31.1,NJ US,40.689531,-74.174462
5,USW00014734,Newark Liberty International Airport,2016-01-06,5.37,0.00,0.0,0.0,28,42,15,230,250.0,12.1,16.1,NJ US,40.689531,-74.174462
6,USW00014734,Newark Liberty International Airport,2016-01-07,3.36,0.00,0.0,0.0,35,46,24,20,360.0,8.9,10.1,NJ US,40.689531,-74.174462
7,USW00014734,Newark Liberty International Airport,2016-01-08,8.05,0.00,0.0,0.0,38,45,31,20,30.0,14.1,16.1,NJ US,40.689531,-74.174462
8,USW00014734,Newark Liberty International Airport,2016-01-09,6.71,0.01,0.0,0.0,44,48,38,60,70.0,13.0,17.0,NJ US,40.689531,-74.174462
9,USW00014734,Newark Liberty International Airport,2016-01-10,15.43,1.77,0.0,0.0,53,65,39,260,270.0,36.0,42.9,NJ US,40.689531,-74.174462


In [6]:
tripdata.head(10)

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,ride_id
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St Path,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Casual,1964,female,018d9bcd-4649-4a72-bde4-9290b037b254
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St Path,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Casual,1962,male,0b6a83c9-fbcb-4c18-8fa8-e789ac12b79d
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St Path,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Casual,1962,female,605e1b1c-ba78-4117-a92e-fd1c0fb121e5
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Casual,1984,male,c9a7c008-92b2-4949-b512-218c9f42c7b5
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Member,<NA>,NaN,b40f9906-64bc-4958-8b0a-95da8b5c8f5b
5,883,2016-01-01 01:03:28,2016-01-01 01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24442,Member,<NA>,NaN,172bb292-46f5-43a4-b0ff-d9cc82c0a080
6,445,2016-01-01 01:07:45,2016-01-01 01:15:11,3186,Grove St Path,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Casual,1988,female,21e3ca1c-4275-428d-a74a-48ce5e581c16
7,192,2016-01-01 01:18:51,2016-01-01 01:22:03,3211,Newark Ave,40.721525,-74.046305,3203,Hamilton Park,40.727596,-74.044247,24625,Casual,1980,male,5889e2f0-5f90-4330-99a5-185927197122
8,409,2016-01-01 01:23:44,2016-01-01 01:30:34,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24429,Casual,1990,male,4ef9cff3-2444-4c79-a99a-aebf261b1eff
9,285,2016-01-01 01:25:12,2016-01-01 01:29:57,3187,Warren St,40.721124,-74.038051,3214,Essex Light Rail,40.712774,-74.036486,24407,Casual,1988,female,38cd1db7-80fd-4aca-8a72-8b030b85059e


In [ ]:
# tripdata.info()

In [ ]:
# weather.info()

In [ ]:
# tripdata.describe()

In [ ]:
# weather.describe()

## Database Design (PostgreSQL)

In [5]:
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.exc import OperationalError
import time

# Define your PostgreSQL database connection parameters
db_config = {
    'user': 'python',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres'
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')
print("Engine created")

table_dict = {
    'tripdata' : 'fact_trip',
    'weather' : 'fact_weather'
}

tables = [tripdata,weather]

inspector = inspect(engine)

for table in tables:

    postgres_name = table_dict[f'{table.Name}']
    pd_name = table.Name

    if not inspector.has_table(postgres_name):
        # Map pandas dtypes to PostgreSQL dtypes
        dtype_mapping = {'object': 'TEXT', 'int64': 'BIGINT', 'float64': 'NUMERIC', 'bool': 'BOOLEAN', 'datetime64': 'TIMESTAMP', 'timedelta': 'INTERVAL'}
        
        # Convert pandas dtypes to PostgreSQL dtypes for each column
        column_types = {col: dtype_mapping.get(str(table[col].dtype), 'TEXT') for col in table.columns}
        
        # Generate the CREATE TABLE SQL statement dynamically based on DataFrame columns and their types
        create_table_sql = f"CREATE TABLE {postgres_name} (\n"
        create_table_sql += ',\n'.join([f"{col} {column_types[col]}" for col in table.columns])
        create_table_sql += "\n);"
        
        # Execute the CREATE TABLE SQL statement
        try:
            start_time = time.time()
            with engine.connect() as connection:
                connection.execute(text(create_table_sql))
                print("Creating {} table in : {} seconds".format(postgres_name,round(time.time() - start_time,2)))
        except OperationalError as e:
            print(f"Error connecting to PostgreSQL: {e}")

        # Write the DataFrame to the PostgreSQL database
        start_time = time.time()
        print("Starting data loading...")
        table.to_sql(postgres_name, engine, index=False, if_exists='replace')
        print("Loading data into table in : {} seconds".format(round(time.time() - start_time,2)))
        
        print(f'DataFrame successfully loaded into the table: {postgres_name} \n')

    else:
        print(f'{postgres_name} already exists in {db_config["database"]}')

Engine created


NameError: name 'tripdata' is not defined

In [ ]:
db_config = {
    'user': 'python',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres'
}

engine = create_engine(f'postgresql://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')
print("Engine created")
print(f'Connected to {db_config["database"]} as user {db_config["user"]}\n')


table_dict = {
    'tripdata' : 'fact_trip',
    'weather' : 'fact_weather'
}

tables = [tripdata,weather]

for table in tables:

    postgres_name = table_dict[f'{table.Name}']

    inspector = inspect(engine)
    print(f'Table {postgres_name} exists:', inspector.has_table(postgres_name))

    if inspector.has_table(postgres_name):
        print('Table exists. Deleting...')
        drop_table_sql = f"DROP TABLE {postgres_name} CASCADE;"
        print(f"Statement used : {drop_table_sql}")
        
        try:
            with engine.connect() as connection:
                # connection.execute("SET search_path TO public, public;")
                connection.execute(text(drop_table_sql))
                # Commit the transaction
                connection.commit()

            print(f'Table {postgres_name} successfully deleted.\n')
        except Exception as e:
            print(f'Error deleting table {postgres_name}: {e}')
            connection.rollback()
            raise  # This will re-raise the exception for more detailed error information

    else:
        print(f'Table {postgres_name} does not exist.\n')